In [3]:
import json
import networkx as nx
from pyvis.network import Network
import matplotlib.pyplot as plt

In [4]:
# No. of agents
N = 3

In [7]:
with open(f'graph_data_N{N:d}.json', 'r') as f:
  struct = json.load(f)
#print(json.dumps(struct, indent = 4))

In [6]:
struct

{'0': {'pattern': ['1', '0', '1'],
  'neigh': [2],
  'strat': {'1': '0', '0': '1'}},
 '1': {'pattern': ['0', '1', '0'],
  'neigh': [2],
  'strat': {'1': '1', '0': '0'}},
 '2': {'pattern': ['1', '0', '0'],
  'neigh': [0, 2],
  'strat': {'11': '0', '00': '0', '10': '1'}}}

In [9]:
edges = []
for n1 in struct:
    for n2 in struct[n1]["neigh"]:  
        if len(struct[n1]["strat"]) > 1:
            edges.append((str(n1), str(n2)))

In [10]:
DG = nx.DiGraph()
DG.add_nodes_from([str(i) for i in range(N)])
DG.add_edges_from(edges)

In [11]:
dnet = Network("400px","400px",notebook=True,directed=True)
dnet.from_nx(DG)
dnet.toggle_physics(True)
dnet.show('dnet.html')

dnet.html


In [54]:
degree = [len(list(DG.neighbors(str(i)))) for i in range(N)]

In [55]:
for sim in range(len(struct["simulations"])):
    # Create graph
    edges = []
    for n1 in struct["simulations"][sim]:
        for n2 in struct["simulations"][sim][n1]["neigh"]:  
            if len(struct["simulations"][sim][n1]["strat"]) > 1:
                edges.append((str(n1), str(n2)))
    DG = nx.DiGraph()
    DG.add_nodes_from([str(i) for i in range(N)])
    DG.add_edges_from(edges)
    
    # Degree of nodes
    degree = [len(list(DG.neighbors(str(i)))) for i in range(N)]